# Module 9: Variant Discovery and Genotyping

## (Re)sequencing Data Flow

1. Raw Reads (FASTQ) --> Mapping/Aligning --> Mapped/Aligned (BAM)
2. Mapped/Aligned (BAM) --> Sample QC --> High-Quality BAMS
3. High-Quality BAMs --> Variant Discovery --> List of Variants (VCF)
4. List of Variants (VCF) --> Variant QC --> High-Quality VCF
5. High-Quality VCF --> Analysis (+ Phenotypes) --> Interpretable Results

We have previously covered the transition from Step 1 to Step 2 (Mapping/Alinging)

## Step 2: Quality Control of NGS Data (BAMs --> High-Quality BAMs)

### Obtainable QC Metrics

#### *Before* Mapping and Alignment (FASTQ)

- Read Depth
    - Approximate only
- Read Length
    - Distribution
- Base Read Quality* (before calibration)
    - Distribution
- K-mer based statistics
    - Is any specific k-mer over- or under- represented?

#### *After* Mapping and Alignment (BAM/CRAM)

- Mapping Quality
    - Distribution
- Duplicate Reads
- Insert Size Distribution
    - Distance between ends of paired-end sequence reads
- Strand (Forward/Reverse)
    - Typical in paired-end mapping, where one is mapped from the forward strand and one from reverse strand
- Adjusted Base Quality
    - Tools provide calculation of base read qualities in more accurate methods, not covered in this lecture
- Reference Bias
    - Reads that are more similar to the reference sequence are "more well mapped," while variants are either less well mapped or not mapped at all
- Contamination

### Selected Metrics in Detail

#### Sequencing Depth

When you order a sequencing service, there is typically a certain number of reads generated from the sequencing machine. A larger number of reads typically costs more. For RNA sequencing, services typically start at 20,000,000 reads.

Whatever service is used, the result is the **total number of base reads.**

The **size of the intended region** is determined by the size of the genome and the intended sequencing. The intended region is different when chosing between DNA or RNA, as well as for the choice of whole genome, exome, or targetted small-region sequencing.

The average sequencing depth is the *total number of base reads* divided by the *size of the intended region*

$$
Average\ sequencing\ depth=\frac{Total\ number\ of\ base\ reads}{size\ of\ intended\ region}
$$

For **human whole genome sequencing**, if we have a FASTQ file with 100 base-long reads (single-ended) with 100,000,000 lines, our average sequencing depth is 0.83.

$$
Total\ number\ of\ base\ reads=\frac{100\ bases\ per\ read\cdot10^{8}\ lines}{4\ lines\ per\ FASTQ\ read}=2.5\cdot10^{9}\ bases \newline
Human\ Genome\ Size=3\cdot10^{9}\ bases \newline
Average\ sequencing\ depth=\frac{2.5\cdot10^{9}\ bases} {3\cdot10^{9}\ bases\ in\ Human\ Genome} = 0.83
$$

For **human whole exome sequencing**, if we have a FASTQ file with 100 base-long reads (single-ended) with 100,000,000 lines, our average sequencing depth is 0.83.

$$
Total\ number\ of\ base\ reads=\frac{100\ bases\ per\ read\cdot10^{8}\ lines}{4\ lines\ per\ FASTQ\ read}=2.5\cdot10^{9}\ bases \newline
Human\ Exome\ Size=1\%\ of\ Human\ Genome = 3\cdot10^{9}\cdot0.01=3\cdot10^{7}\ bases \newline
Average\ sequencing\ depth=\frac{2.5\cdot10^{9}\ bases} {3\cdot10^{7}\ bases\ in\ Human\ Exome} = 83
$$

#### Sequencing/PCR Duplicates

PCR duplicates reads during sample prep, causing many identical reads from the same source sequence. Optical duplicates, which are read by the same cluster twice on the sequencer, can be particularly problematic since the sequencer might treat them as separate sequences.

High duplication can lead to problems in downstream analysis, such as causing a skew to allele frequencies. It can make it difficult to assess if alleles are variants or not.

It's important to try to mark your duplicates and take them into account in analysis. We want as many *independent observations* as possible.

### Programs and Tools

#### FASTQC for *Before* Mapping and Alignment

A [free Java program](http://www.bioinformatics.babraham.ac.uk/projects/fastqc/) that reports the quality profile of reads in FASTQ or SAM/BAM files, but mostly used on FASTQ files. There is also a web interface!

FASTQC identifies several QC metrics and issues. It reports basic statistics and Kmer content, but also additional information.

Per Base:
- Sequence Quality
- Sequence Content
- GC Content
- N Content

Per Sequence:
- Sequence Quality Scores
- Sequence GC content
- Sequence Length Distribution
- Sequence Duplication Levels
- Overrepresented Sequences

Remember, later sequences are more prone to diminishing quality of the reads!

#### Visualization for *After* Mapping and Alignment

Most useful for examining a specific region of interest, but not best for looking at a whole genome

#### Picard

A Java-based command-line utility tool that can be used to manipulate SAM/BAM files. Some of the command-line utilities include:
- `CollectInsertSizeMetrics`
- `CollectGcBiasMetrics`
- `CollectAlignmentSummaryMetrics`
- `QualityScoreDistribution`
- `MarkDuplicates`
    - Most commonly used to remove duplicate reads

#### RNA-SeQC

A Picard-like tool for RNA sequencing data, available [online](https://software.broadinstitute.org/cancer/cga/rna-seqc). It was actually derived from Picard.

## Step 3: (High-Quality BAMS --> Variant Discovery --> List of Variants (VCF))

### Background

#### History of Human Genetic Variation

Revelations about Human Genetic Variation, such as SNPs and structural genomic variation, were overall considered the 2007 Scientific Breakthrough of the Year.

Timeline:
- 1997: Sanger sequencing first developed
- 1990 - 2003: Human genome Project
    - Goal to discover all human genes and make them accessible for further biological study
    - Goal to discover the complete sequence of the human reference genome
    - Used Sanger Sequencing with targeted genotyping
- 2001: First human reference genome published
- 2002: Genome-Wide Genotyping (GWAS) first developed
- 2002 - 2005: International HapMap Project
    - Goal to develop a haplotype map of the human genome that described the common patterns of human DNA sequence variation
    - Discovered millions of SNPs, and developed arrays to test for the distributions of SNPs to examine haplotype differences
    - Used GWAS
- 2005: NGS 
- 2008: Individual sequencing became more widely available
- 2008 - 2015: 1000 Genomes Project
    - Goal to establish a catalogue of human genetic variation through whole genome sequencing of 1000 anonymous participants across different ethnic groups
- 2009: Exome sequencing (WES) first developed

##### Highlighted Projects

###### International HapMap project

The International HapMap Project was from 2002-2005, and involved international cooperation to use analyses of the distributions of SNPs to develop a haplotype map of common patterns of human DNA sequence variation.

Four populations, and a total of 270 individuals, were characterized to create these maps:
- CEU: CEPH 
    -Utah residents with ancestry from Northern and Western Europe
    - 30 trios
- CHB: Beijing, China
    - Han Chinese
    - 45 individuals
- JPT: Tokyo, Japan
    - Japanese
    - 45 individuals
- YRI: Ibadan, Nigeria
    - Yoruba
    - 30 trios

###### 1000 Genomes Project

The 1000 Genomes Project was from 2008-2015, and involved international cooperation to to perform whole genome sequencing and complete description of the genetic diversity in 1000 individuals from multiple world populations.

The price of sequencing dropped so dramatically during the study, that they were able to sequence well over 1000 individuals.

#### Forms of Genome Variation

There are *over 4 million DNA variants per human individual*. This is how each of us have unique DNA, and there is so much variation between individuals even in the same species.

Common forms of Genome Variation include:
- Single Nucleotide Variants
    - Substitution at a single nucleotide
    - Only a Single Nucleotide Polymorphism (SNP) if shared by 1% of the population
- Multi-Nucleotide Variants
    - INDELs
        - Insertions and Deletions of less than 50 base pairs
        - Can cause a frameshift mutation
    - Microsatellites or Short Tandem Repeats (STRs)
        - A tract of tandemly repeated DNA motifs between 1-10 nuleotides in length that are repeated 5-50 times
        - Commonly used for linkage analysis due to high variability in the repeat number between individuals
    - Large Copy Number Variants (CNVs)
        - A structural variation
        - A duplication or deletion that changes the number of copies of a particular DNA segment within a genome
    - Mobile Element Insertions (MEIs)
        - A structural variation
        - Occurs when a segement of DNA is "copied" and "pasted" into different locations in the genome through an RNA intermediate
    - Inversions
        - A structural variation
        - Occurs when a portion of a chromosome breaks off and reattaches in the reverse orientation on the *same chromosome*
        - Portions of DNA may or may not be lost in this process
    - Translocations
        - A structural variation
        - Occurs when a portion of a chromosome breaks off and reattaches to a *different chromosome*
    - Aneuploidy
        - A structural variation
        - Occurs when there are either extra or missing chromosomes, leading to unbalanced chromosome counts

### Finding Genomic Variation

At its core, genomic variation is any difference from the reference. We can look at the results of our alignment and mapping to identify these differences.

#### Genotyping

**Genotyping** is the process of determining genetic differences between individuals by using a set of *known markers*. This is in contrast to **sequencing,** as sequencing is the process of determining the full nucleotide order of a DNA sequence that involves *previously unknown* variants.

Genotyping depends on the understanding of DNA as being composed of alleles. Each **allele** is one of a number of alternative forms of the same genetic locus/structure. These alleles may be characterized by their SNPs or other variations. 

**Allele frequencies** reflect the proportion of the population that has that specific allele; these frequencies range from 0, where the allele is not expressed at all in the population, to 1, where every individual in the population has that allele, without exceptions.

A **Minor Allele Frequency (MAF)** refers to the fraction of an allele that is *not* in the majority of samples; these frequencies are similar to *allele frequencies,* but instead range from 0 to 0.5.

##### Array Genotyping

Genotyping frequently uses *genome chip arrays*. Original chips could only test for 500K SNPs, but modern chips can test for over 2 Million SNPs and even some INDELs at one time. Each chip is rather inexpensive.

Linkage disequilibrium is a key component of genotyping, as it indicates where alleles occur together more often than can be accounted for by change alone due to their physical proximity on a chromosome.

This works by using correlation structures at specific markers of the genome to make inferences on what is between those specific markers.

Array genotyping typically involves thousands of samples being processed at once, and the results can produce graphs demonstrating the distributions of alleles.

For a gene with two alleles you might have "Intensity (A)", which is produced from Allele A (and could be represented by red) and "Intensity (B)", which is produced by Allele B (and could be represented by blue). 

In an **Illumina plot**, Intensity (A) may be on the X-axis, and Intensity (B) may be on the Y-axis:
- Individual samples with AA alleles would have a high Intensity (A), would be red, and clustered along the X-axis with low values on the Y-axis
- Individual samples with BB alleles would have a high Intensity (B), would be blue, clustered along the Y-axis with low values on the X-axis 
- Individual samples with AB alleles would be purple and scattered linearly between the X and Y axes

In an **AffyMetrix plot**, Intensity (A) and Intensity (B) may be represented on a normalized plot where values of Intensity (A) is on the left of the X-axis, values of Intensity (B) is on the right of the X-axis, and overall intensity is represented on the Y-axis:
- Individual samples with AA alleles would have a high Intensity (A), would be red, and clustered to the left of the X-axis
- Individual samples with BB alleles would have a high Intensity (B), would be blue, clustered along to the right of the X-axis
- Individual samples with AB alleles would be purple, and clustered between values of Intensity (A) and Intensity (B) on the X-axis, in the middle.

Regardless of plot, the clusters of each allele are usually clearly separated.

#### Sequencing for Variant Discovery

Sequencing is not quite as clear as geneotyping because we are looking at individual nucleotides and may have read errors. As such, we cannot typically say with *absolute certainty* that there is a variant, but we are able to express a probability that the variant exists.

However, we can identify *previously unknown* variants this way.

##### Phred Score and SNPs

A single nucleotide variation is not typically enough to declare that the variation is a SNP due to read errors. However, if that same allele occurs multiple times at the same locus, then there is stronger evidence that it is a SNP.

Base read quality is an important metric to assessing if a substitution is a true SNP. If alleles with the variation have similar or better Phred scores than those that are identical to the reference, that's support that it's a true SNP.

Bases with a Phred less than 20 are typically represeted with lowercase letters, to make comparison easier.

##### Base Read Distribution and Genotype Likelihood

**Genotype likelihood**, P(D|G), is the likelihood of observing a distribution of base reads, given the underling (true) genotype.

**Posterior probability of genotype**, P(G|D) is the probability of each genotype given the distribution of the base reads.

As a result, SNP and INDEL calling are large-scale **Bayesian Modeling Problems**

###### Ordered Model

$$

P(G|D)=\frac{P(G)\cdot P(D|G)}{\sum_i{P(G_i)\cdot P(D|G_i)} } \newline
P(D|G) = \Pi_j(\frac{P(D_j|H_1)}{2}+\frac{P(D_j|H_2)}{2}) , where\ G=H_1H_2 (Diploid\ Assumption)
\newline
P(D|H) is\ the\ haploid\ likelihood\ function

$$

The **inference** asks what the genotype of G of a sample, when given read data D.

Bayes' rule allows us to calculate the probability of each possible G.

Product expansion assumes that reads are independent, and it relies on a likelihood function to estimate the probability of sample data given a proposed haplotype.

SNP Genotype likelihoods then translate this into:

$$
P(D_j|H)=P(D_j|b), [single\ base\ pileup]
\newline
P(D_j|b)= \begin{cases}
   1-\in_j &\ D_j=b, \\
   \in_j & otherwise
\end{cases}
$$

If *b* is the "true base" in the haplotype, the probability of observing the "true base" is the converse of the probability of a read error, and the probability of reading anything else is the probability of a read error.

All diploid genotypes (AA, AC,...,GT,TT) are considered at each base.

The likelihood of genotype is computed using only pileup of bases and associated quality scores at a given locus. Only "good bases" are included.

"Good bases" are those that satisfy a minimum base quality, mapping read quality, pair mapping quality, and NQS.

For an example:

D (read sequence) = {C,C,C,T,T}, all with a Phred score of 20 (1% error rate)

$$
P(D_j|b)= \begin{cases}
   1-0.01 &\ D_j=b, \\
   0.01 & otherwise
\end{cases}
$$

1. What is the Genotype likelihood of observing "C", when the true underlying genotype is "C"?

$$
P(D_j|b)=P("C"|"C") = 0.99
$$

2. What is the genotype likelihood of observing "C", when the true underling genotype is "T"?

$$
P(D_j|b)=P("C"|"T") = 0.01
$$

3. What is the genotype likelihood of observing "C", when the true underlying genotype is "A"?

$$
P(D_j|b)=P("C"|"C") = 0.99
$$

4. What is the genotype likelihood of observing "C" if the genotype is "CT"?

$$
P(D_j|b)=P("C"|"CT") = \frac{ P("C"|"C") + P("C"|"T")  }{2} = \frac{0.99 + 0.01}{2} = \frac{1}{2} = 0.50
$$

If we take the diploid likelihood for every diploid pair in the genotype and multiply them together, we will get the overall probability of the observed genotype "CCCTT"

$$
P(D|G) = P("C"|"CC")\cdot P("C"|"CC")\cdot P("C"|"CT") \cdot P("T"|"TT")
$$

There is also an unordered model, but it is more complicated if the reads have different Phred scores.

##### INDELs

Insertions and deletions that are typically less than 50bp in size (thus relatively short).

Genotype accuracy for INDELs is usually lower than accuracy for SNPs and even SVs, as they are senstitive to mapping and alignment artifacts.

It is important to check to see if the INDEL is from a well-sequenced region and isn't from some sort of repreat region or a centromere or telomere.

### Software and Tools for Variant Calling

- SAMTOOLS/BCFTOOLS
- FreeBayes
- VT
    - Developed by the University of Michigan
    - Used by the NCBI TOPMed project
- GATK
    - Developed by the Broad Institute 
    - Possibly the most commonly used variant calling software in the world
- and more!

#### Variant Call Format (VCF)

Variant Call Format (VCF) files are tab-delimited text files, often compressed, which contain lists of genomic variations.

Each row represents a variant, and each column represents a sample.

There is a header at the top of the file that provides information about each column and its contents. Each header line starts with `##`. The last row of the header only starts with `#`, and acts as "title" of each column.

The first eight columns provide the summary information about the variant itself:
- `CHROM` has the chromosome number
- `POS` gives the position
- `ID` gives an ID for the variant
- `REF` gives the reference base in that specific position in the reference genome
- `ALT` gives the base present in the variant 
- `QUAL` gives the quality metric but does not define what the metric is
- `FILTER` is an optional flag which can be defined in the header such as failing to meet a quality threshold
- `INFO` provides additional information as defined in the header
- `FORMAT` indicates how the information in the sample columns will be formatted as defined in the header.

The minimum content in the `FORMAT` of the sample columns is Genotype (`GT`):
- 0: `REF`erence allele
- 1: first `ALT` allele
- 2: seocond `ALT` allele, and so on.
- Diploids are either Phased or Unphased
    - Phased (`0|1`)
    - Unphased(`1|1`)